**[xhlulu](https://www.kaggle.com/xhlulu) <br>
[Copied From Here](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification/discussion/204950)**

In [ ]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

tf.config.experimental.list_physical_devices()

In [ ]:
#Decoder for Image
def build_decoder(with_labels = True , size=300):
        
        def decode(path):
            
            file_path = tf.io.read_file(path)
            img = tf.image.decode_jpeg(file_path, channels=3)
            
            img = tf.cast(img, tf.float32) / 255.0
            img = tf.image.resize(img,(size,size))
            
            return img
        
        def decode_with_labels(path, label):
            return decode(path), label
        
        return decode_with_labels if with_labels else decode

def build_augmenter(with_labels = True):
    def img_aug(img):
        img = tf.image.random_jpeg_quality(img,75,95)
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        img = tf.image.random_brightness(img,0.1)
        return img
    def aug_with_labels(img,labels):
        return img_aug(img) , labels
    
    return aug_with_labels if with_labels else img_aug(img)

#Builder for Dataset    
def builder(target,labels=None,bsize=16,decoder=None,augment=True,cache_dir="",cache=True):
    
    if cache_dir != "" and cache == True:
        os.makedirs(cache_dir,exist_ok=True)
        
    if decoder is None :
        decoder = build_decoder(decoder is not None)
    if augment is True :
        augmenter = build_augmenter(with_labels = True)
    auto = tf.data.experimental.AUTOTUNE
    slices = target if labels is None else (target , labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decoder , num_parallel_calls=auto)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augmenter, num_parallel_calls=auto) if augment else dset
    dset = dset.batch(bsize).prefetch(auto)
    
    return dset

In [ ]:
train_dir = r"/kaggle/input/ranzcr-clip-catheter-line-classification/train/"
test_dir = r"/kaggle/input/ranzcr-clip-catheter-line-classification/test/"

train_df = pd.read_csv("/kaggle/input/ranzcr-clip-catheter-line-classification/train.csv")
sample_df = pd.read_csv("/kaggle/input/ranzcr-clip-catheter-line-classification/sample_submission.csv")
print("Train Shape : ",train_df.shape)
print(train_df.nunique())
train_df.head()

In [ ]:
#Change StudyInstanceUID to match file name for flow_from_dataframe
if not ".jpg" in train_df.iloc[0,0]:
    train_df.iloc[:,0] = train_dir + train_df["StudyInstanceUID"] + ".jpg"

sample_target = test_dir + sample_df["StudyInstanceUID"] + ".jpg"
shuffle(train_df)
input_size = [224,240,360,300,380,456,528,600]
labels = train_df.columns[1:-1]
img_size = input_size[4]
batch = 16
random = 13

print(train_df.loc[0,"StudyInstanceUID"])

In [ ]:
train_x , valid_x , train_y , valid_y = train_test_split(train_df.iloc[:,0],np.array(train_df.iloc[:,1:-1]),test_size=.2,random_state=random)
print(train_x[0])
print(train_y[0])

In [ ]:
train_decode = build_decoder(with_labels=True,size=img_size)
test_decode = build_decoder(with_labels=False , size=img_size)
c_dir = "/kaggle/tf_cache"

train = builder(train_x,labels=train_y,bsize=batch,decoder=train_decode,cache_dir=c_dir,cache=True)
valid = builder(valid_x,labels=valid_y,bsize=batch,decoder=train_decode,augment=False,cache_dir=c_dir, cache=True)
test = builder(sample_target,bsize=batch,decoder=test_decode,augment=False,cache=False)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
import tensorflow.keras.applications.efficientnet as efn

In [ ]:
weight_path = "../input/tfkeras-efficientnet-weights/efficientnetb4_notop.h5"
model = Sequential([efn.EfficientNetB4(input_shape=(img_size, img_size, 3),weights=weight_path
                    ,include_top=False,drop_connect_rate=0.6),                    
                tf.keras.layers.GlobalAveragePooling2D(),
                tf.keras.layers.Dense(len(labels), activation='sigmoid') ])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=[tf.keras.metrics.AUC(multi_label=True)],run_eagerly=None)
model.summary()

In [ ]:
#from tensorflow.keras.utils import plot_model
#plot_model(model,show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)

In [ ]:
#Create Callback
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model_b4.h5', save_best_only=True, monitor='val_auc', mode='max')

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc", patience=3, min_lr=1e-6, mode='max')

callbacks_list=[checkpoint,lr_reducer]

In [ ]:
history = model.fit(train,validation_data=valid,steps_per_epoch=len(train),epochs=20
                    ,callbacks=callbacks_list,verbose=1)

In [ ]:
model.load_weights('model_b4.h5')
model.save("model_b4.h5")

In [ ]:
hist_df = pd.DataFrame(history.history)
hist_df.to_csv('history.csv')

In [ ]:
sample_df[labels] = model.predict(test, verbose=1)
sample_df.to_csv('submission.csv', index=False)

sample_df.head()